In [ ]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import numpy as np
import soundfile
import torch
from IPython.display import Audio
from torch.nn import functional as F

In [ ]:
from myddsp.preprocessors import LegacyLoudness

In [ ]:
y, _ = librosa.load("../data/test/samples/violin-1.wav", sr=48000, mono=False)

In [ ]:
Audio(y, rate=48000)

In [ ]:
y_trimmed, _ = librosa.effects.trim(y, top_db=80)

In [ ]:
Audio(y_trimmed, rate=48000)

In [ ]:
ld = LegacyLoudness()

In [ ]:
yy = torch.from_numpy(y_trimmed)
yy = yy.unsqueeze(0)
amps1 = ld(yy).flatten().numpy()

In [ ]:
def randomize_phase(x):
    s = torch.fft.rfft(x)
    s_ = torch.polar(s.abs(), torch.rand_like(s.angle()) * 2.0 * np.pi)
    x_ = torch.fft.irfft(s_)

    return x_

In [ ]:
shit = randomize_phase(yy)

In [ ]:
yy.shape

In [ ]:
amps = ld(yy).flatten().numpy()

In [ ]:
all(np.isclose(amps, amps1))

In [ ]:
plt.plot(amps)

In [ ]:
plt.plot(amps1)

In [ ]:
import einops
from torchaudio.transforms import Loudness

import myddsp.constants as C

In [ ]:
tl = Loudness(48000)

In [ ]:
reframed = librosa.util.frame(y_trimmed, frame_length=19200, hop_length=C.HOP_LENGTH)

In [ ]:
reframed.shape

In [ ]:
reframed.shape

In [ ]:
rf = torch.from_numpy(np.copy(reframed))

In [ ]:
c, n, f = rf.shape
rf_ = einops.rearrange(rf, "c n f -> f c n")
win = torch.hann_window(n)
rf_ = einops.einsum(rf_, win, "f c n, n -> f c n")
start = (19200 - n) // 2
end = (19200 - n) - start
rf_ = F.pad(rf_, (start, end))
amp = tl(rf_)

In [ ]:
rf_.shape

In [ ]:
plt.plot(amp.numpy())

In [ ]:
plt.plot(amp.numpy())

In [ ]:
amp.indices()